In [1]:
import json

from deb_ai_tor.environments import DebateSandwichEnv

In [2]:
agents = ["agent_0", "agent_1"]

env = DebateSandwichEnv(agents=agents)
obs = env.reset(options={
    "question": "What is a debate sandwich?",
    "answers": ["Arguments with answers in the middle", "A sandwich eaten while debating", "Answers with arguments in the middle"],
    "num_rounds": 3,
})

print(json.dumps(obs, indent=4))

{
    "agent_0": {
        "message": "Which of the following answers do you think is best?\n",
        "info": "Question: What is a debate sandwich?\nAnswers: {'A': 'Arguments with answers in the middle', 'B': 'A sandwich eaten while debating', 'C': 'Answers with arguments in the middle'}\n"
    },
    "agent_1": {
        "message": "Which of the following answers do you think is best?\n",
        "info": "Question: What is a debate sandwich?\nAnswers: {'A': 'Arguments with answers in the middle', 'B': 'A sandwich eaten while debating', 'C': 'Answers with arguments in the middle'}\n"
    }
}


In [3]:
actions = {
    "agent_0": "A",
    "agent_1": "C",
}
obs, rewards, terminations, truncations, infos = env.step(actions)
print(json.dumps(obs, indent=4))

{
    "agent_0": {
        "message": "Debate your choice of answer\n",
        "info": {
            "agent_0": "A",
            "agent_1": "C"
        }
    },
    "agent_1": {
        "message": "Debate your choice of answer\n",
        "info": {
            "agent_0": "A",
            "agent_1": "C"
        }
    }
}


In [4]:
actions = {
    "agent_0": "Really weak arguments for A.",
    "agent_1": "Strong arguments for C.",
}
obs, rewards, terminations, truncations, infos = env.step(actions)
print(json.dumps(obs, indent=4))

{
    "agent_0": {
        "message": "Which of the following answers do you now think is best?\n",
        "info": {
            "agent_0": "Really weak arguments for A.",
            "agent_1": "Strong arguments for C."
        }
    },
    "agent_1": {
        "message": "Which of the following answers do you now think is best?\n",
        "info": {
            "agent_0": "Really weak arguments for A.",
            "agent_1": "Strong arguments for C."
        }
    }
}


In [5]:
actions = {
    "agent_0": "C",
    "agent_1": "C",
}
obs, rewards, terminations, truncations, infos = env.step(actions)
print("Rewards:", rewards)
print(f"{env.initial_answers=}", f"{env.final_answers=}")

Rewards: {'agent_0': -1, 'agent_1': 1}
env.initial_answers={'agent_0': 'A', 'agent_1': 'C'} env.final_answers={'agent_0': 'C', 'agent_1': 'C'}
